In [1]:
from asynch import *
import tensorflow as tf
from utils import create_session_config
from models.utils import corr
import numpy as np
import tensorflow_probability as tp
FLAGS.per_process_gpu_memory_fraction=0.01
from data.registry import etf2d
from models.nn_pdf_common import transform_x, create_cdf_layer_partial_monotonic_MLP, create_pdf_layer_mv,create_monotone_dense_layer
from models.utils import extract_xy, metric_loglikelihood, train_op, constrain_cdf, print_tensor
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
%matplotlib inline

/home/pawel/tensorflow/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/pawel/tensorflow/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/pawel/tensorflow/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/pawel/tensorflow/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/pawel/tensorflow/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/pawel/tensorflow/lib/python3.5/importli

In [2]:
def create_positive_weights(shape, positive_transform):
    w=tf.get_variable(name="weights-raw", shape=shape, dtype=tf.float32)

    if positive_transform == "exp":
        w_positive = tf.exp(w, name='weights-positive')
    elif positive_transform == "square":
        w_positive = tf.square(w, name='weights-positive')
    elif positive_transform == "softplus":
        w_positive = tf.nn.softplus(w, name='weights-positive')
    else:
        raise ValueError("wrong positive_transform: %s"%positive_transform)

    return w_positive 

def create_bias(units):
    with tf.name_scope('biases'):
        b=tf.get_variable(shape=(units,), dtype=tf.float32, name="bias")
    return b

def create_monotone_dense_layer(input, units, activation, positive_transform):
    w = create_positive_weights(shape=[input.shape[-1].value, units], positive_transform=positive_transform)
#     w = tf.constant(np.full([input.shape[-1].value, units], 1.0), dtype=tf.float32)
    b = create_bias(units)
    res = tf.matmul(input, w)+ b
#     
    return res if activation is None else activation(res)

def run(data): 
    d=tf.placeholder(shape=(None,2),dtype=tf.float32)
    x=tf.slice(d,[0,0],[-1,1])
    y=tf.slice(d,[0,1],[-1,1])
    
    
    l = tf.concat([x,y],axis=1)
    
    
#     l=tf.concat([x,y], axis=1)
#     l = tf.layers.dense(l, 10, activation=tf.nn.tanh)
#     l = tf.layers.dense(l, 10, activation=tf.nn.tanh)
#     l = tf.layers.dense(l, 1)
#     for i, size in enumerate([10]):
#         with tf.variable_scope("l%d" % (i + 1)):
#             l=create_monotone_dense_layer(l, size, activation=tf.nn.tanh, positive_transform="square")
    
    l=create_monotone_dense_layer(l, 1, activation=tf.nn.tanh, positive_transform="square")
    
    tvars = tf.trainable_variables()
    

#     l=x+y+x*y*4
#     l=tf.reduce_sum(l, axis=1)
    
    g = tf.gradients(l, x)[0]
    g=tf.gradients(g, y)[0]
    
    with tf.Session(config=create_session_config()) as sess:
        sess.run(tf.global_variables_initializer())
        return sess.run([g,tvars], feed_dict={d:data}) , [var.name for var in tvars] 

(g,var_vals), names = invoke_in_process_pool(1, Callable(run, [[-1,-2],[3,4]]))[0]

dict(zip(names, var_vals))
print("gradient: %r" % g)

{'bias:0': array([1.1592973], dtype=float32)}

gradient: array([[ 1.8230022e-01],
       [-7.1525557e-07]], dtype=float32)


In [50]:
x=1
y=2
-2*np.tanh(x+y)*(1-np.tanh(x+y)**2)

-0.019634494363042477